In [2]:
import pandas as pd
import os
import numpy as np
import statistics

In [2]:
# 데이터 폴더 경로
folder_path = r"C:\Users\user\Desktop\2024-2\기계학습\mon_50"

In [3]:
X1 = [] # Array to store instances (timestamps) - 19,000 instances, e.g., [[0.0, 0.5, 3.4, ...], [0.0, 4.5, ...], [0.0, 1.5, ...], ... [... ,45.8]]
X2 = [] # Array to store instances (direction*size) - size information
y = [] # Array to store the site of each instance - 19,000 instances, e.g., [0, 0, 0, 0, 0, 0, ..., 94, 94, 94, 94, 94]



for file_name in os.listdir(folder_path):
    if 'split' in file_name:  # split 파일만 선택
        class_label = int(file_name.split('-')[0])  # 클래스 번호 추출
        file_path = os.path.join(folder_path, file_name)
        if os.path.getsize(file_path) > 0:  # 빈 파일 확인
            df = pd.read_csv(file_path, sep='\t', header=None)
            X1.append(df[0])  # 타임스탬프
            X2.append(df[2])  # 방향*크기
            y.append(class_label)  # 클래스 라벨
size = len(y)

print(f"총 데이터 포인트: {len(X1)}")
print(len(X1))
print(len(X2))
print(len(y))


print(X2[0:1000])

총 데이터 포인트: 45583
45583
45583
45583
[0    -512.0
1    -512.0
2    -512.0
3    -512.0
4    -512.0
5    -512.0
6    -512.0
7    -512.0
8    -512.0
9    -512.0
10   -512.0
11   -512.0
12   -512.0
13   -512.0
14   -512.0
15   -512.0
16   -512.0
17   -512.0
18   -512.0
19   -512.0
20   -512.0
21   -512.0
22   -512.0
23   -512.0
24   -512.0
25   -512.0
26   -512.0
27   -512.0
28   -512.0
29   -512.0
30   -512.0
31   -512.0
32   -512.0
33   -512.0
34   -512.0
35   -512.0
36    512.0
37    512.0
38    512.0
39    512.0
40    512.0
41    512.0
42    512.0
43    512.0
44    512.0
45    512.0
46    512.0
47    512.0
48    512.0
49    512.0
50    512.0
Name: 2, dtype: float64, 0     -512.0
1     -512.0
2     -512.0
3     -512.0
4     -512.0
       ...  
303   -512.0
304   -512.0
305   -512.0
306   -512.0
307   -512.0
Name: 2, Length: 308, dtype: float64, 0     -512.0
1     -512.0
2     -512.0
3     -512.0
4     -512.0
       ...  
207   -512.0
208   -512.0
209   -512.0
210   -512.0
211   -512.0
Nam

In [4]:
num_total_packets = [] # feature 1
sum_packets = [] # feature 2
num_incoming_packets = [] # feature 3
frac_incoming_packets = [] # feature 4
num_outgoing_packets=[] # feature 5
frac_outgoing_packets=[] # feature 6

for size_seq in X2:
    # 1. Total number of pacekts
    total = len(size_seq)
    num_total_packets.append(total)

    # 2. Sum of packets (absolute value gives the size regardless of direction)
    sum_pckts = sum(abs(size) for size in size_seq)
    sum_packets.append(sum_pckts)

    # 3. Number of incoming packets
    incoming = sum(1 for size in size_seq if size < 0)
    num_incoming_packets.append(incoming)

    # 4. Number of incoming packets as a fraction of the total number of packets
    frac_incoming = incoming / total if total > 0 else 0
    frac_incoming_packets.append(frac_incoming)

    # 5. Number of outgoing packets
    outgoing = sum(1 for size in size_seq if size > 0)
    num_outgoing_packets.append(outgoing)

    # 6. Number of outgoing packets as a fraction of the total number of packets
    frac_outgoing = outgoing / total if total > 0 else 0
    frac_outgoing_packets.append(frac_outgoing)

data = {
    'feature 1': num_total_packets,
    'feature 2': sum_packets,
    'feature 3': num_incoming_packets,
    'feature 4': frac_incoming_packets,
    'feature 5': num_outgoing_packets,
    'feature 6': frac_outgoing_packets
}

df = pd.DataFrame(data)
print(df.head())

#%%
print(len(num_total_packets))
print(len(sum_packets))
print(len(num_incoming_packets))
print(len(frac_incoming_packets))
print(len(num_outgoing_packets))
print(len(frac_outgoing_packets))

   feature 1  feature 2  feature 3  feature 4  feature 5  feature 6
0         51    26112.0         36   0.705882         15   0.294118
1        308   157696.0        308   1.000000          0   0.000000
2        212   108544.0        212   1.000000          0   0.000000
3        517   264704.0        411   0.794971        106   0.205029
4        333   170496.0        333   1.000000          0   0.000000
45583
45583
45583
45583
45583
45583


In [5]:
packet_size_ratios = [] # feature 7
burst_lengths = []  # feature 8

for size_seq in X2:

    # 7. Packet Size Ratios
    outgoing_size_sum = sum(size for size in size_seq if size > 0)
    incoming_size_sum = abs(sum(size for size in size_seq if size < 0))  # 절대값 사용
    packet_size_ratio = outgoing_size_sum / incoming_size_sum if incoming_size_sum > 0 else 0
    packet_size_ratios.append(packet_size_ratio)

    # 8. Burst Lengths
    bursts = []
    current_burst = 0
    size_seq = size_seq.values if isinstance(size_seq, pd.Series) else size_seq  # Pandas Series 처리
    last_direction = np.sign(size_seq[0]) if len(size_seq) > 0 else 0  # 초기 방향 설정

    for size in size_seq:
        direction = np.sign(size)
        if direction == last_direction:
            current_burst += size
        else:
            bursts.append(current_burst)
            current_burst = size          # 새 버스트 시작
            last_direction = direction

    if current_burst != 0:
        bursts.append(current_burst)  # 마지막 버스트 추가

    burst_lengths.append(len(bursts))      # 버스트 길이 저장

#%%
# 리스트 길이 확인
print(f"Length of packet_size_ratios: {len(packet_size_ratios)}")
print(f"Length of burst_lengths: {len(burst_lengths)}")

# 데이터프레임 생성
if len(packet_size_ratios) == len(burst_lengths):
    data = {
        'feature 7': packet_size_ratios,
        'feature 8': burst_lengths,
    }
    df = pd.DataFrame(data)
    print(df.head())
else:
    print("Error: Mismatched lengths between features.")

Length of packet_size_ratios: 45583
Length of burst_lengths: 45583
   feature 7  feature 8
0   0.416667          2
1   0.000000          1
2   0.000000          1
3   0.257908         82
4   0.000000          1


In [6]:
average_incoming_ordering=[] # feature 9
std_dev_incoming_ordering=[] # feature 10
average_outgoing_ordering=[] # feature 11
std_dev_outgoing_ordering=[] # feature 12

for size_seq in X2:
    # 9. Average of the incoming packet ordering list
    incoming_ordering_list = []
    for idx, size in enumerate(size_seq):
        if size < 0:
            incoming_ordering_list.append(idx)

    average_incoming_ordering.append(np.mean(incoming_ordering_list))

    # 10. Standard deviation of the incoming packet ordering list
    std_dev_incoming_ordering.append(np.std(incoming_ordering_list))

    # 11. Average of the outgoing packet ordering list
    outgoing_ordering_list = []
    for idx, size in enumerate(size_seq):
        if size > 0:
            outgoing_ordering_list.append(idx)
    average_outgoing_ordering.append(np.mean(outgoing_ordering_list))

    # 12. Standard deviation of the outgoing packet ordering list
    std_dev_outgoing_ordering.append(np.std(outgoing_ordering_list))

    data = {
    'feature 9': average_incoming_ordering,
    'feature 10': std_dev_incoming_ordering,
    'feature 11': average_outgoing_ordering,
    'feature 12': std_dev_outgoing_ordering
}

df = pd.DataFrame(data)
print(df.head())

#%%
print(len(num_total_packets))
print(len(sum_packets))
print(len(num_incoming_packets))
print(len(frac_incoming_packets))
print(len(num_outgoing_packets))
print(len(frac_outgoing_packets))
print(len(packet_size_ratios))
print(len(burst_lengths))
print(len(average_incoming_ordering))
print(len(std_dev_incoming_ordering))
print(len(average_outgoing_ordering))
print(len(std_dev_outgoing_ordering))

C:\Users\user\AppData\Roaming\Python\Python38\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\user\AppData\Roaming\Python\Python38\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\user\AppData\Roaming\Python\Python38\site-packages\numpy\core\_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\user\AppData\Roaming\Python\Python38\site-packages\numpy\core\_methods.py:226: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
C:\Users\user\AppData\Roaming\Python\Python38\site-packages\numpy\core\_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


   feature 9  feature 10  feature 11  feature 12
0   17.50000   10.388295   43.000000    4.320494
1  153.50000   88.911473         NaN         NaN
2  105.50000   61.198448         NaN         NaN
3  266.79562  141.040872  223.896226  173.334333
4  166.00000   96.128386         NaN         NaN
45583
45583
45583
45583
45583
45583
45583
45583
45583
45583
45583
45583


In [7]:
# Feature 리스트 초기화
avg_packet_size = []  # feature 13
max_packet_size = []  # feature 14
avg_inter_arrival_time = []  # feature 15
var_packet_size = []  # feature 16
alternative_packets_per_second_sum = []  # feature 17

for time_seq, size_seq in zip(X1, X2):  # X1과 X2는 각각 시간과 크기 리스트들

    # Pandas Series가 들어올 경우 리스트 또는 Numpy 배열로 변환
    if isinstance(size_seq, pd.Series):
        size_seq = size_seq.tolist()
    if isinstance(time_seq, pd.Series):
        time_seq = time_seq.tolist()

    # 13. Average Packet Size
    avg_size = sum(abs(size) for size in size_seq) / len(size_seq) if size_seq else 0
    avg_packet_size.append(avg_size)

    # 14. Max Packet Size
    max_size = max(abs(size) for size in size_seq) if size_seq else 0
    max_packet_size.append(max_size)

    # 15. Packet Inter-Arrival Time (IAT)
    inter_arrival_time = np.mean(np.diff(time_seq)) if len(time_seq) > 1 else 0
    avg_inter_arrival_time.append(inter_arrival_time)

    # 16. Variance of Packet Size
    var_size = np.var([abs(size) for size in size_seq]) if size_seq else 0
    var_packet_size.append(var_size)

    # 17. Alternative Packets Per Second
    packets_per_subset = max(len(size_seq) // 20, 1)  # 최소 크기 설정
    subset_sums = []

    for i in range(0, len(size_seq), packets_per_subset):
        start_index = i
        end_index = min(i + packets_per_subset, len(size_seq))

        # Time interval 계산
        time_interval = time_seq[end_index - 1] - time_seq[start_index]
        time_interval = max(time_interval, 0.001)  # Zero division 방지

        packets_per_second = (end_index - start_index) / time_interval
        subset_sums.append(packets_per_second)

    alternative_packets_per_second_sum.append(sum(subset_sums))

# DataFrame 생성
data = {
    'feature 13': avg_packet_size,
    'feature 14': max_packet_size,
    'feature 15': avg_inter_arrival_time,
    'feature 16': var_packet_size,
    'feature 17': alternative_packets_per_second_sum,
}

df = pd.DataFrame(data)

# 출력 및 검증
print(df.head())
print("Lengths of Features:")
print(len(avg_packet_size))
print(len(max_packet_size))
print(len(avg_inter_arrival_time))
print(len(var_packet_size))
print(len(alternative_packets_per_second_sum))

   feature 13  feature 14  feature 15  feature 16    feature 17
0       512.0       512.0    0.102538         0.0  36540.278097
1       512.0       512.0    0.006914         0.0  31655.348907
2       512.0       512.0    0.020005         0.0  26786.998182
3       512.0       512.0    0.013309         0.0  15513.805971
4       512.0       512.0    0.016218         0.0  18302.241692
Lengths of Features:
45583
45583
45583
45583
45583


In [8]:
num_packets_per_second = []  # feature 18
mean_packets_per_second = []  # feature 19
std_packets_per_second = []  # feature 20
med_packets_per_second = []  # feature 21

for i in range(len(X1)):
    time_seq = X1[i]

    # time_seq가 pandas.Series일 경우 리스트로 변환
    if isinstance(time_seq, pd.Series):
        time_seq = time_seq.tolist()

    if time_seq:  # time_seq가 비어있지 않은 경우에만 처리
        num_packets = len(time_seq) / time_seq[-1] if time_seq[-1] > 0 else 0
        mean_packets = statistics.mean(time_seq)
        std_packets = statistics.stdev(time_seq) if len(time_seq) > 1 else 0
        med_packets = statistics.median(time_seq)
    else:  # 비어 있는 경우 기본값 설정
        num_packets = 0
        mean_packets = 0
        std_packets = 0
        med_packets = 0

    num_packets_per_second.append(num_packets)
    mean_packets_per_second.append(mean_packets)
    std_packets_per_second.append(std_packets)
    med_packets_per_second.append(med_packets)

# DataFrame 생성
data = {
    'feature 18': num_packets_per_second,
    'feature 19': mean_packets_per_second,
    'feature 20': std_packets_per_second,
    'feature 21': med_packets_per_second,
}

df = pd.DataFrame(data)

# 출력 및 검증
print(df.head())
print("Lengths of Features:")
print(len(num_packets_per_second))
print(len(mean_packets_per_second))
print(len(std_packets_per_second))
print(len(med_packets_per_second))

   feature 18  feature 19  feature 20  feature 21
0    9.947533    1.253829    1.891627    0.005500
1  145.111896    0.455774    0.532438    0.260500
2   50.225065    3.223250    1.049663    3.167000
3   75.282921    4.323082    1.037928    4.051427
4   61.844182    0.902940    1.083641    0.256000
Lengths of Features:
45583
45583
45583
45583


In [9]:
first_packet_direction = []  # feature 22
num_packets_longest_burst = []  # feature 23

for size_seq in X2:
    # size_seq가 pandas.Series일 경우 리스트로 변환
    if isinstance(size_seq, pd.Series):
        size_seq = size_seq.tolist()

    # 초기화
    bursts = 1 if size_seq else 0
    current_direction = size_seq[0] if size_seq else None
    current_burst_length = 1
    longest_burst = 1

    outgoing_counts = []
    burst_packet_counts = []

    # Feature 22: First Packet Direction
    first_direction = 1 if size_seq and size_seq[0] > 0 else -1 if size_seq and size_seq[0] < 0 else 0
    first_packet_direction.append(first_direction)

    # Burst 계산
    for i in range(1, len(size_seq)):
        if size_seq[i] * current_direction <= 0:  # 방향 변경 시
            bursts += 1
            outgoing_counts.append(sum(1 for size in size_seq[i - current_burst_length:i] if size > 0))
            burst_packet_counts.append(current_burst_length)
            longest_burst = max(longest_burst, current_burst_length)
            current_direction = size_seq[i]
            current_burst_length = 1
        else:
            current_burst_length += 1

    if size_seq:
        # 마지막 버스트 처리
        outgoing_counts.append(sum(1 for size in size_seq[-current_burst_length:] if size > 0))
        burst_packet_counts.append(current_burst_length)
        longest_burst = max(longest_burst, current_burst_length)

    # Feature 23: Number of Packets in the Longest Burst 저장
    num_packets_longest_burst.append(longest_burst)

    # DataFrame 생성
data = {
    'feature 22': first_packet_direction,
    'feature 23': num_packets_longest_burst,
    }

df = pd.DataFrame(data)

# 출력 및 검증
print(df.head())

   feature 22  feature 23
0          -1          36
1          -1         308
2          -1         212
3           1          40
4          -1         333


In [10]:
mon_data_23 = {
    'num_total_packets': num_total_packets,
    'sum_packets': sum_packets,
    'num_incoming_packets': num_incoming_packets,
    'frac_incoming_packets': frac_incoming_packets,
    'num_outgoing_packets': num_outgoing_packets,
    'frac_outgoing_packets': frac_outgoing_packets,
    'packet_size_ratios': packet_size_ratios,
    'burst_lengths': burst_lengths,
    'average_incoming_ordering': average_incoming_ordering,
    'std_dev_incoming_ordering': std_dev_incoming_ordering,
    'average_outgoing_ordering': average_outgoing_ordering,
    'std_dev_outgoing_ordering': std_dev_outgoing_ordering,
    'avg_packet_size':avg_packet_size,
    'max_packet_size':max_packet_size,
    'avg_inter_arrival_time':avg_inter_arrival_time,
    'var_packet_size':var_packet_size,
    'alternative_packets_per_second_sum':alternative_packets_per_second_sum,
    'num_packets_per_second':num_packets_per_second,
    'mean_packets_per_second':mean_packets_per_second,
    'std_packets_per_second':std_packets_per_second,
    'med_packets_per_second':med_packets_per_second,
    'first_packet_direction':first_packet_direction,
    'num_packets_longest_burst':num_packets_longest_burst
}

df_23 = pd.DataFrame(mon_data_23)
print(df_23.head())

label = {'label': y}
df_label = pd.DataFrame(label)

print(len(df_label))
print(df_label.head())

   num_total_packets  sum_packets  num_incoming_packets  \
0                 51      26112.0                    36   
1                308     157696.0                   308   
2                212     108544.0                   212   
3                517     264704.0                   411   
4                333     170496.0                   333   

   frac_incoming_packets  num_outgoing_packets  frac_outgoing_packets  \
0               0.705882                    15               0.294118   
1               1.000000                     0               0.000000   
2               1.000000                     0               0.000000   
3               0.794971                   106               0.205029   
4               1.000000                     0               0.000000   

   packet_size_ratios  burst_lengths  average_incoming_ordering  \
0            0.416667              2                   17.50000   
1            0.000000              1                  153.50000   
2    

In [11]:
df_23.to_csv(r"C:\Users\user\Desktop\2024-2\기계학습\extra_mon50_23.csv", index=False)
df_label.to_csv(r"C:\Users\user\Desktop\2024-2\기계학습\extra_mon50_label.csv", index=False)

unmon

In [3]:
folder_path = r"C:\Users\user\Desktop\2024-2\기계학습\unmon_10000"

X1 = [] # Array to store instances (timestamps) - 19,000 instances, e.g., [[0.0, 0.5, 3.4, ...], [0.0, 4.5, ...], [0.0, 1.5, ...], ... [... ,45.8]]
X2 = [] # Array to store instances (direction*size) - size information
y = [] # Array to store the site of each instance - 19,000 instances, e.g., [0, 0, 0, 0, 0, 0, ..., 94, 94, 94, 94, 94]


# Iterate over the files and process only those containing valid values
for file_name in os.listdir(folder_path):
    if 'split' in file_name:  # Select only split files
        class_label = int(file_name.split('_')[0])  # Extract class number
        file_path = os.path.join(folder_path, file_name)

        if os.path.getsize(file_path) > 0:  # Check if file is non-empty
            df = pd.read_csv(file_path, sep='\t', header=None)

            X1.append(df[0])  # Append timestamps (column 0)
            X2.append(df[2])  # Append direction*size (column 2)
            y.append(class_label)  # Append class label

# Size of the resulting data
size = len(y)

print(f"총 데이터 포인트: {len(X1)}")
print(len(X1))
print(len(X2))
print(len(y))

y = [-1] * len(y)
#%%
print(X1[0:1000])

총 데이터 포인트: 44910
44910
44910
44910
[0    0.0000
1    0.0040
2    0.0055
3    0.0065
4    0.9715
Name: 0, dtype: float64, 0      0.0000
1      0.1310
2      0.1311
3      0.2485
4      0.2486
        ...  
120    5.4400
121    5.4405
122    5.5425
123    5.5460
124    5.5485
Name: 0, Length: 125, dtype: float64, 0        0.0000
1        0.1030
2        0.1720
3        2.9370
4        5.7730
         ...   
1485    13.5920
1486    13.5935
1487    13.8495
1488    13.8530
1489    13.8540
Name: 0, Length: 1490, dtype: float64, 0      0.0000
1      0.0070
2      0.0100
3      0.0100
4      0.0120
        ...  
297    2.3855
298    2.3855
299    2.3895
300    2.3895
301    2.3895
Name: 0, Length: 302, dtype: float64, 0       0.000000
1       0.028000
2       0.031000
3       0.517028
4       0.518028
          ...   
1016    2.955103
1017    2.969103
1018    2.988200
1019    3.230103
1020    7.586626
Name: 0, Length: 1021, dtype: float64, 0      1.387779e-17
1      6.900000e-02
2      2.83400

In [4]:
print(y[100:200])

[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]


In [15]:
num_total_packets = [] # feature 1
sum_packets = [] # feature 2
num_incoming_packets = [] # feature 3
frac_incoming_packets = [] # feature 4
num_outgoing_packets=[] # feature 5
frac_outgoing_packets=[] # feature 6

for size_seq in X2:
    # 1. Total number of pacekts
    total = len(size_seq)
    num_total_packets.append(total)

    # 2. Sum of packets (absolute value gives the size regardless of direction)
    sum_pckts = sum(abs(size) for size in size_seq)
    sum_packets.append(sum_pckts)

    # 3. Number of incoming packets
    incoming = sum(1 for size in size_seq if size < 0)
    num_incoming_packets.append(incoming)

    # 4. Number of incoming packets as a fraction of the total number of packets
    frac_incoming = incoming / total if total > 0 else 0
    frac_incoming_packets.append(frac_incoming)

    # 5. Number of outgoing packets
    outgoing = sum(1 for size in size_seq if size > 0)
    num_outgoing_packets.append(outgoing)

    # 6. Number of outgoing packets as a fraction of the total number of packets
    frac_outgoing = outgoing / total if total > 0 else 0
    frac_outgoing_packets.append(frac_outgoing)

data = {
    'feature 1': num_total_packets,
    'feature 2': sum_packets,
    'feature 3': num_incoming_packets,
    'feature 4': frac_incoming_packets,
    'feature 5': num_outgoing_packets,
    'feature 6': frac_outgoing_packets
}

df = pd.DataFrame(data)
print(df.head())

print(len(num_total_packets))
print(len(sum_packets))
print(len(num_incoming_packets))
print(len(frac_incoming_packets))
print(len(num_outgoing_packets))
print(len(frac_outgoing_packets))

   feature 1  feature 2  feature 3  feature 4  feature 5  feature 6
0          5     2560.0          5   1.000000          0   0.000000
1        125    64000.0         96   0.768000         29   0.232000
2       1490   762880.0       1490   1.000000          0   0.000000
3        302   154624.0        302   1.000000          0   0.000000
4       1021   522752.0        974   0.953967         47   0.046033
44910
44910
44910
44910
44910
44910


In [16]:
packet_size_ratios = [] # feature 7
burst_lengths = []  # feature 8

for size_seq in X2:

    # 7. Packet Size Ratios
    outgoing_size_sum = sum(size for size in size_seq if size > 0)
    incoming_size_sum = abs(sum(size for size in size_seq if size < 0))  # 절대값 사용
    packet_size_ratio = outgoing_size_sum / incoming_size_sum if incoming_size_sum > 0 else 0
    packet_size_ratios.append(packet_size_ratio)

    # 8. Burst Lengths
    bursts = []
    current_burst = 0
    size_seq = size_seq.values if isinstance(size_seq, pd.Series) else size_seq  # Pandas Series 처리
    last_direction = np.sign(size_seq[0]) if len(size_seq) > 0 else 0  # 초기 방향 설정

    for size in size_seq:
        direction = np.sign(size)
        if direction == last_direction:
            current_burst += size
        else:
            bursts.append(current_burst)
            current_burst = size          # 새 버스트 시작
            last_direction = direction

    if current_burst != 0:
        bursts.append(current_burst)  # 마지막 버스트 추가

    burst_lengths.append(len(bursts))      # 버스트 길이 저장

#%%
# 리스트 길이 확인
print(f"Length of packet_size_ratios: {len(packet_size_ratios)}")
print(f"Length of burst_lengths: {len(burst_lengths)}")

# 데이터프레임 생성
if len(packet_size_ratios) == len(burst_lengths):
    data = {
        'feature 7': packet_size_ratios,
        'feature 8': burst_lengths,
    }
    df = pd.DataFrame(data)
    print(df.head())
else:
    print("Error: Mismatched lengths between features.")

Length of packet_size_ratios: 44910
Length of burst_lengths: 44910
   feature 7  feature 8
0   0.000000          1
1   0.302083         41
2   0.000000          1
3   0.000000          1
4   0.048255          7


In [17]:
average_incoming_ordering=[] # feature 9
std_dev_incoming_ordering=[] # feature 10
average_outgoing_ordering=[] # feature 11
std_dev_outgoing_ordering=[] # feature 12

for size_seq in X2:
    # 9. Average of the incoming packet ordering list
    incoming_ordering_list = []
    for idx, size in enumerate(size_seq):
        if size < 0:
            incoming_ordering_list.append(idx)

    average_incoming_ordering.append(np.mean(incoming_ordering_list))

    # 10. Standard deviation of the incoming packet ordering list
    std_dev_incoming_ordering.append(np.std(incoming_ordering_list))

    # 11. Average of the outgoing packet ordering list
    outgoing_ordering_list = []
    for idx, size in enumerate(size_seq):
        if size > 0:
            outgoing_ordering_list.append(idx)
    average_outgoing_ordering.append(np.mean(outgoing_ordering_list))

    # 12. Standard deviation of the outgoing packet ordering list
    std_dev_outgoing_ordering.append(np.std(outgoing_ordering_list))

    data = {
    'feature 9': average_incoming_ordering,
    'feature 10': std_dev_incoming_ordering,
    'feature 11': average_outgoing_ordering,
    'feature 12': std_dev_outgoing_ordering
}

df = pd.DataFrame(data)
print(df.head())

#%%
print(len(num_total_packets))
print(len(sum_packets))
print(len(num_incoming_packets))
print(len(frac_incoming_packets))
print(len(num_outgoing_packets))
print(len(frac_outgoing_packets))
print(len(packet_size_ratios))
print(len(burst_lengths))
print(len(average_incoming_ordering))
print(len(std_dev_incoming_ordering))
print(len(average_outgoing_ordering))
print(len(std_dev_outgoing_ordering))

C:\Users\user\AppData\Roaming\Python\Python38\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\user\AppData\Roaming\Python\Python38\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\user\AppData\Roaming\Python\Python38\site-packages\numpy\core\_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\user\AppData\Roaming\Python\Python38\site-packages\numpy\core\_methods.py:226: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
C:\Users\user\AppData\Roaming\Python\Python38\site-packages\numpy\core\_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


    feature 9  feature 10  feature 11  feature 12
0    2.000000    1.414214         NaN         NaN
1   69.854167   35.015467   36.000000   25.944237
2  744.500000  430.125854         NaN         NaN
3  150.500000   87.179413         NaN         NaN
4  489.858316  281.808104  927.404255  242.307055
44910
44910
44910
44910
44910
44910
44910
44910
44910
44910
44910
44910


In [18]:
# Feature 리스트 초기화
avg_packet_size = []  # feature 13
max_packet_size = []  # feature 14
avg_inter_arrival_time = []  # feature 15
var_packet_size = []  # feature 16
alternative_packets_per_second_sum = []  # feature 17

for time_seq, size_seq in zip(X1, X2):  # X1과 X2는 각각 시간과 크기 리스트들

    # Pandas Series가 들어올 경우 리스트 또는 Numpy 배열로 변환
    if isinstance(size_seq, pd.Series):
        size_seq = size_seq.tolist()
    if isinstance(time_seq, pd.Series):
        time_seq = time_seq.tolist()

    # 13. Average Packet Size
    avg_size = sum(abs(size) for size in size_seq) / len(size_seq) if size_seq else 0
    avg_packet_size.append(avg_size)

    # 14. Max Packet Size
    max_size = max(abs(size) for size in size_seq) if size_seq else 0
    max_packet_size.append(max_size)

    # 15. Packet Inter-Arrival Time (IAT)
    inter_arrival_time = np.mean(np.diff(time_seq)) if len(time_seq) > 1 else 0
    avg_inter_arrival_time.append(inter_arrival_time)

    # 16. Variance of Packet Size
    var_size = np.var([abs(size) for size in size_seq]) if size_seq else 0
    var_packet_size.append(var_size)

    # 17. Alternative Packets Per Second
    packets_per_subset = max(len(size_seq) // 20, 1)  # 최소 크기 설정
    subset_sums = []

    for i in range(0, len(size_seq), packets_per_subset):
        start_index = i
        end_index = min(i + packets_per_subset, len(size_seq))

        # Time interval 계산
        time_interval = time_seq[end_index - 1] - time_seq[start_index]
        time_interval = max(time_interval, 0.001)  # Zero division 방지

        packets_per_second = (end_index - start_index) / time_interval
        subset_sums.append(packets_per_second)

    alternative_packets_per_second_sum.append(sum(subset_sums))

# DataFrame 생성
data = {
    'feature 13': avg_packet_size,
    'feature 14': max_packet_size,
    'feature 15': avg_inter_arrival_time,
    'feature 16': var_packet_size,
    'feature 17': alternative_packets_per_second_sum,
}

df = pd.DataFrame(data)

# 출력 및 검증
print(df.head())
print("Lengths of Features:")
print(len(avg_packet_size))
print(len(max_packet_size))
print(len(avg_inter_arrival_time))
print(len(var_packet_size))
print(len(alternative_packets_per_second_sum))

   feature 13  feature 14  feature 15  feature 16    feature 17
0       512.0       512.0    0.242875         0.0   5000.000000
1       512.0       512.0    0.044746         0.0   7946.929639
2       512.0       512.0    0.009304         0.0  15804.471792
3       512.0       512.0    0.007939         0.0  28323.605894
4       512.0       512.0    0.007438         0.0  21153.226318
Lengths of Features:
44910
44910
44910
44910
44910


In [19]:
num_packets_per_second = []  # feature 18
mean_packets_per_second = []  # feature 19
std_packets_per_second = []  # feature 20
med_packets_per_second = []  # feature 21

for i in range(len(X1)):
    time_seq = X1[i]

    # time_seq가 pandas.Series일 경우 리스트로 변환
    if isinstance(time_seq, pd.Series):
        time_seq = time_seq.tolist()

    if time_seq:  # time_seq가 비어있지 않은 경우에만 처리
        num_packets = len(time_seq) / time_seq[-1] if time_seq[-1] > 0 else 0
        mean_packets = statistics.mean(time_seq)
        std_packets = statistics.stdev(time_seq) if len(time_seq) > 1 else 0
        med_packets = statistics.median(time_seq)
    else:  # 비어 있는 경우 기본값 설정
        num_packets = 0
        mean_packets = 0
        std_packets = 0
        med_packets = 0

    num_packets_per_second.append(num_packets)
    mean_packets_per_second.append(mean_packets)
    std_packets_per_second.append(std_packets)
    med_packets_per_second.append(med_packets)

# DataFrame 생성
data = {
    'feature 18': num_packets_per_second,
    'feature 19': mean_packets_per_second,
    'feature 20': std_packets_per_second,
    'feature 21': med_packets_per_second,
}

df = pd.DataFrame(data)

# 출력 및 검증
print(df.head())
print("Lengths of Features:")
print(len(num_packets_per_second))
print(len(mean_packets_per_second))
print(len(std_packets_per_second))
print(len(med_packets_per_second))

   feature 18  feature 19  feature 20  feature 21
0    5.146680    0.197500    0.432686    0.005500
1   22.528611    3.663683    1.327970    3.943500
2  107.550166   12.056758    1.328478   12.654000
3  126.386273    0.906094    1.041532    0.212250
4  134.578935    1.277876    0.637777    1.092028
Lengths of Features:
44910
44910
44910
44910


In [20]:
first_packet_direction = []  # feature 22
num_packets_longest_burst = []  # feature 23

for size_seq in X2:
    # size_seq가 pandas.Series일 경우 리스트로 변환
    if isinstance(size_seq, pd.Series):
        size_seq = size_seq.tolist()

    # 초기화
    bursts = 1 if size_seq else 0
    current_direction = size_seq[0] if size_seq else None
    current_burst_length = 1
    longest_burst = 1

    outgoing_counts = []
    burst_packet_counts = []

    # Feature 22: First Packet Direction
    first_direction = 1 if size_seq and size_seq[0] > 0 else -1 if size_seq and size_seq[0] < 0 else 0
    first_packet_direction.append(first_direction)

    # Burst 계산
    for i in range(1, len(size_seq)):
        if size_seq[i] * current_direction <= 0:  # 방향 변경 시
            bursts += 1
            outgoing_counts.append(sum(1 for size in size_seq[i - current_burst_length:i] if size > 0))
            burst_packet_counts.append(current_burst_length)
            longest_burst = max(longest_burst, current_burst_length)
            current_direction = size_seq[i]
            current_burst_length = 1
        else:
            current_burst_length += 1

    if size_seq:
        # 마지막 버스트 처리
        outgoing_counts.append(sum(1 for size in size_seq[-current_burst_length:] if size > 0))
        burst_packet_counts.append(current_burst_length)
        longest_burst = max(longest_burst, current_burst_length)

    # Feature 23: Number of Packets in the Longest Burst 저장
    num_packets_longest_burst.append(longest_burst)

# DataFrame 생성
data = {
    'feature 22': first_packet_direction,
    'feature 23': num_packets_longest_burst,
}

df = pd.DataFrame(data)

# 출력 및 검증
print(df.head())

   feature 22  feature 23
0          -1           5
1          -1          27
2          -1        1490
3          -1         302
4           1         965


In [21]:
unmon_data_23 = {
    'num_total_packets': num_total_packets,
    'sum_packets': sum_packets,
    'num_incoming_packets': num_incoming_packets,
    'frac_incoming_packets': frac_incoming_packets,
    'num_outgoing_packets': num_outgoing_packets,
    'frac_outgoing_packets': frac_outgoing_packets,
    'packet_size_ratios': packet_size_ratios,
    'burst_lengths': burst_lengths,
    'average_incoming_ordering': average_incoming_ordering,
    'std_dev_incoming_ordering': std_dev_incoming_ordering,
    'average_outgoing_ordering': average_outgoing_ordering,
    'std_dev_outgoing_ordering': std_dev_outgoing_ordering,
    'avg_packet_size':avg_packet_size,
    'max_packet_size':max_packet_size,
    'avg_inter_arrival_time':avg_inter_arrival_time,
    'var_packet_size':var_packet_size,
    'alternative_packets_per_second_sum':alternative_packets_per_second_sum,
    'num_packets_per_second':num_packets_per_second,
    'mean_packets_per_second':mean_packets_per_second,
    'std_packets_per_second':std_packets_per_second,
    'med_packets_per_second':med_packets_per_second,
    'first_packet_direction':first_packet_direction,
    'num_packets_longest_burst':num_packets_longest_burst
}

df_23 = pd.DataFrame(unmon_data_23)
print(df_23.head())

label = {'label': y}
df_label = pd.DataFrame(label)

print(len(df_label))
print(df_label.head())

   num_total_packets  sum_packets  num_incoming_packets  \
0                  5       2560.0                     5   
1                125      64000.0                    96   
2               1490     762880.0                  1490   
3                302     154624.0                   302   
4               1021     522752.0                   974   

   frac_incoming_packets  num_outgoing_packets  frac_outgoing_packets  \
0               1.000000                     0               0.000000   
1               0.768000                    29               0.232000   
2               1.000000                     0               0.000000   
3               1.000000                     0               0.000000   
4               0.953967                    47               0.046033   

   packet_size_ratios  burst_lengths  average_incoming_ordering  \
0            0.000000              1                   2.000000   
1            0.302083             41                  69.854167   
2    

In [ ]:
df_23.to_csv(r"C:\Users\user\Desktop\2024-2\기계학습\extra_unmon10000_23.csv", index=False)
df_label.to_csv(r"C:\Users\user\Desktop\2024-2\기계학습\extra_unmon10000_label.csv", index=False)

In [5]:
label = {'label': y}
df_label = pd.DataFrame(label)

print(len(df_label))
print(df_label.head())

44910
   label
0     -1
1     -1
2     -1
3     -1
4     -1


In [ ]:
df_label.to_csv(r"C:\Users\user\Desktop\2024-2\기계학습\extra_unmon10000_label.csv", index=False)